# Radical Pilot Tutorial


## 1. Installation

    pip install radical.pilot
    pip install radical.utils
    
## 2. Radical Pilot Setup

Import dependency and create callbacks for state update

Documentation: http://radicalpilot.readthedocs.org/en/latest/machconf.html#preconfigured-resources

Mongo:
    
           monogodb
           $ db.createCollection("sc15-tutorial", {capped: true, size: 100000, max: 100});

In [1]:
import os, sys
import commands
import radical.pilot as rp

os.environ["RADICAL_PILOT_DBURL"]="mongodb://localhost/radicalpilot"

#os.environ["RADICAL_PILOT_DBURL"]="mongodb://129.114.108.124:27017/sctut15"
os.environ["RADICAL_PILOT_VERBOSE"]="DEBUG"

""" DESCRIPTION: Tutorial 1: A Simple Workload consisting of a Bag-of-Tasks
"""

def pilot_state_cb (pilot, state):
    if not pilot:
        return
    print "[Callback]: ComputePilot '%s' state: %s." % (pilot.uid, state)
    if state == rp.FAILED:
        sys.exit (1)

def unit_state_cb (unit, state):
    if not unit:
        return

    global CNT
    print "[Callback]: unit %s on %s: %s." % (unit.uid, unit.pilot_id, state)
    if state == rp.FAILED:
        print "stderr: %s" % unit.stderr
        sys.exit(2)

In [2]:
commands.getstatusoutput('echo $RADICAL_PILOT_VERBOSE')

(0, 'DEBUG')

## 3. Submit Pilot

In [3]:
# Create a new session. No need to try/except this: if session creation
# fails, there is not much we can do anyways...
session = rp.Session()
print "session id: %s" % session.uid
# all other pilot code is now tried/excepted.  If an exception is caught, we
# can rely on the session object to exist and be valid, and we can thus tear
# the whole RP stack down via a 'session.close()' call in the 'finally'
# clause...

try:
    # ----- CHANGE THIS -- CHANGE THIS -- CHANGE THIS -- CHANGE THIS ------
    # 
    # Change the user name below if you are using a remote resource 
    # and your username on that resource is different from the username 
    # on your local machine. 
    #
    c = rp.Context('ssh')
    c.user_id = "tg824689"
    session.add_context(c)
    # Add a Pilot Manager. Pilot managers manage one or more ComputePilots.
    print "Initializing Pilot Manager ..."
    pmgr = rp.PilotManager(session=session)
    # Register our callback with the PilotManager. This callback will get
    # called every time any of the pilots managed by the PilotManager
    # change their state.
    pmgr.register_callback(pilot_state_cb)
    # ----- CHANGE THIS -- CHANGE THIS -- CHANGE THIS -- CHANGE THIS ------
    # 
    # If you want to run this example on your local machine, you don't have 
    # to change anything here. 
    # 
    # Change the resource below if you want to run on a remote resource. 
    # You also might have to set the 'project' to your allocation ID if 
    # your remote resource does compute time accounting. 
    #
    # A list of preconfigured resources can be found at: 
    # http://radicalpilot.readthedocs.org/en/latest/machconf.html#preconfigured-resources
    # 
    pdesc = rp.ComputePilotDescription ()
    pdesc.resource = "local.localhost"  # NOTE: This is a "label", not a hostname
    pdesc.runtime  = 10 # minutes
    pdesc.cores    = 16
    pdesc.cleanup  = False
    pdesc.project  = 'TG-MCB090174'
    pdesc.queue    = 'development'
    # submit the pilot.
    print "Submitting Compute Pilot to Pilot Manager ..."
    pilot = pmgr.submit_pilots(pdesc)
    # Combine the ComputePilot, the ComputeUnits and a scheduler via
    # a UnitManager object.
    print "Initializing Unit Manager ..."
    umgr = rp.UnitManager (session=session,
                           scheduler=rp.SCHED_DIRECT_SUBMISSION)
    # Register our callback with the UnitManager. This callback will get
    # called every time any of the units managed by the UnitManager
    # change their state.
    umgr.register_callback(unit_state_cb)
    # Add the created ComputePilot to the UnitManager.
    print "Registering Compute Pilot with Unit Manager ..."
    umgr.add_pilots(pilot)
    NUMBER_JOBS  = 64 # the total number of cus to run
    # submit CUs to pilot job
    cudesc_list = []
    for i in range(NUMBER_JOBS):
        # -------- BEGIN USER DEFINED CU DESCRIPTION --------- #
        cudesc = rp.ComputeUnitDescription()
        cudesc.environment = {'CU_NO': i}
        cudesc.executable  = "/bin/echo"
        cudesc.arguments   = ['I am CU number $CU_NO']
        cudesc.cores       = 1
        # -------- END USER DEFINED CU DESCRIPTION --------- #
        cudesc_list.append(cudesc)
    # Submit the previously created ComputeUnit descriptions to the
    # PilotManager. This will trigger the selected scheduler to start
    # assigning ComputeUnits to the ComputePilots.
    print "Submit Compute Units to Unit Manager ..."
    cu_set = umgr.submit_units (cudesc_list)
    print "Waiting for CUs to complete ..."
    umgr.wait_units()
    print "All CUs completed successfully!"
    for cu in cu_set:
        print cu.stdout
        
except Exception as e:
    # Something unexpected happened in the pilot code above
    print "caught Exception: %s" % e
    raise
    
except (KeyboardInterrupt, SystemExit) as e:
    # the callback called sys.exit(), and we can here catch the
    # corresponding KeyboardInterrupt exception for shutdown.  We also catch
    # SystemExit (which gets raised if the main threads exits for some other
    # reason).
    print "need to exit now: %s" % e
    
finally:
    # always clean up the session, no matter if we caught an exception or
    # not.
    print "closing session"
    session.close ()

    # the above is equivalent to
    #
    #   session.close (cleanup=True, terminate=True)
    #
    # it will thus both clean out the session's database record, and kill
    # all remaining pilots (none in our example).

session id: rp.session.LMUCX29608.local.luckow.016733.0013
Initializing Pilot Manager ...
Submitting Compute Pilot to Pilot Manager ...
[Callback]: ComputePilot 'pilot.0000' state: PendingLaunch.
Initializing Unit Manager ...
Registering Compute Pilot with Unit Manager ...
Submit Compute Units to Unit Manager ...
[Callback]: ComputePilot 'pilot.0000' state: Launching.
Waiting for CUs to complete ...
[Callback]: unit unit.000032 on pilot.0000: PendingInputStaging.
[Callback]: unit unit.000055 on pilot.0000: PendingInputStaging.
[Callback]: unit unit.000030 on pilot.0000: PendingInputStaging.
[Callback]: unit unit.000031 on pilot.0000: PendingInputStaging.
[Callback]: unit unit.000036 on pilot.0000: PendingInputStaging.
[Callback]: unit unit.000015 on pilot.0000: PendingInputStaging.
[Callback]: unit unit.000052 on pilot.0000: PendingInputStaging.
[Callback]: unit unit.000053 on pilot.0000: PendingInputStaging.
[Callback]: unit unit.000010 on pilot.0000: PendingInputStaging.
[Callback]: 

2015-10-25 18:30:32,540: radical.pilot       : MainProcess                     : PilotLauncherWorker-1: ERROR   : Using bootstrapper /Users/luckow/anaconda/lib/python2.7/site-packages/radical/pilot/bootstrapper/default_bootstrapper.sh
Copying bootstrapper 'file://localhost/Users/luckow/anaconda/lib/python2.7/site-packages/radical/pilot/bootstrapper/default_bootstrapper.sh' to agent sandbox (<saga.filesystem.directory.Directory object at 0x10781d550>).
Copying sdist 'file://localhost/Users/luckow/anaconda/lib/python2.7/site-packages/radical/utils/radical.utils-v0.37-master.tar.gz' to sandbox (file://localhost/Users/luckow/radical.pilot.sandbox/rp.session.LMUCX29608.local.luckow.016733.0013-pilot.0000/).
Copying sdist 'file://localhost/Users/luckow/anaconda/lib/python2.7/site-packages/saga/saga-python-0.37.tar.gz' to sandbox (file://localhost/Users/luckow/radical.pilot.sandbox/rp.session.LMUCX29608.local.luckow.016733.0013-pilot.0000/).
Pilot launching failed! (copy (file://localhost/Use